In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [107]:
caminho_pai = "planilhas/tabela_cliente.xlsx"

In [108]:
df = pd.DataFrame(pd.read_excel(caminho_pai)) 

In [109]:
cod = 25

In [110]:
df = df.astype('object')

#df['CÓDIGO'] = df['CÓDIGO'].astype(float)

In [111]:
df.dtypes

CÓDIGO                  object
NOME                    object
CPF                     object
RG                      object
CONTA CONTRATO          object
ENDEREÇO                object
CEP                     object
MUNICÍPIO               object
UF                      object
RAMO DE ATIVIDADE       object
CLASSE                  object
TIPO DE LIGAÇÃO         object
DISJUNTOR DE ENTRADA    object
TIPO DE RAMAL           object
ENQUADRAMENTO           object
X                       object
Y                       object
dtype: object

In [112]:
df_filtrado = df[(df['CÓDIGO']==cod)]

In [113]:
df_filtrado.dtypes

CÓDIGO                  object
NOME                    object
CPF                     object
RG                      object
CONTA CONTRATO          object
ENDEREÇO                object
CEP                     object
MUNICÍPIO               object
UF                      object
RAMO DE ATIVIDADE       object
CLASSE                  object
TIPO DE LIGAÇÃO         object
DISJUNTOR DE ENTRADA    object
TIPO DE RAMAL           object
ENQUADRAMENTO           object
X                       object
Y                       object
dtype: object

In [114]:
dict1 = df_filtrado.to_dict(orient='index')[0]

In [115]:
for key in dict1:
        if isinstance(dict1[key], str) == False:
            dict1[key] = str(dict1[key])

dict1['CÓDIGO'] = int(dict1['CÓDIGO'])

In [116]:
dict1

{'CÓDIGO': 25,
 'NOME': 'JOSYNELMA ROCHA SANTANA',
 'CPF': '88412385349',
 'RG': '37369938',
 'CONTA CONTRATO': '3013136814',
 'ENDEREÇO': 'R. NOVA, 21, RES TUPINAMBA, TURU',
 'CEP': '65066350',
 'MUNICÍPIO': 'SÃO LUÍS',
 'UF': 'MA',
 'RAMO DE ATIVIDADE': 'RESIDENCIAL',
 'CLASSE': 'RESIDENCIAL',
 'TIPO DE LIGAÇÃO': 'MONOFÁSICO',
 'DISJUNTOR DE ENTRADA': '40',
 'TIPO DE RAMAL': 'AÉREO',
 'ENQUADRAMENTO': 'INDIVIDUAL',
 'X': '-2.523146',
 'Y': '-44.223851'}

In [72]:
','.join(dict1.get('ENDEREÇO').split(',')[0:-1])

'R. NOVA, 21, RES TUPINAMBA'

In [46]:
wb_path = "planilhas/relatorio_comissionamento.xlsx"
wb = load_workbook(wb_path)
ws = wb.active

In [50]:
ws['C6'] = dict1.get('NOME')
ws['O6'] = dict1.get('CONTA CONTRATO')
ws['C9'] = dict1.get('CEP')
ws['F9'] = dict1.get('MUNICÍPIO')
ws['C8'] = dict1.get('ENDEREÇO')

In [66]:
primeiro_nome = dict1.get('NOME').split(' ')[0].lower()

In [67]:
primeiro_nome

'josynelma'

In [52]:
wb.save(f'planilhas/RELATORIO_COMISSIONAMENTO_{primeiro_nome}.xlsx')

In [73]:
wb.close()

In [74]:
wb_path = "planilhas/solicitacao_acesso.xlsx"
wb = load_workbook(wb_path)
ws = wb.active

In [55]:
ws['C10'] = dict1.get('NOME')
ws['P27'] = dict1.get('DISJUNTOR DE ENTRADA')
ws['F29'] = dict1.get('TIPO DE RAMAL')

In [56]:
wb.save(f'planilhas/SOLICITACAO_ACESSO_{primeiro_nome}.xlsx')